In [ ]:
from torch.utils.data import Dataset
import json

class TextData(Dataset):
    
    def __init__(self,path:str,tokenizer,print_data=False):
        self.data = json.load(open(path,'r')) 
        self.X = []
        
        for i in self.data:
            for j in i['dailog']:
                self.X.append(j['text'])
                
        for idx,i in enumerate(self.X):
            try:
                self.X[idx] = "<sos>" + i + "<model>" + self.X[idx+1] + "<eos>"
            except:
                break
        
        self.X = self.X[:5000]
        if print_data==True:
            print(self.X[0])
            
        self.X_encoded = tokenizer(self.X,max_length=50,padding=True,truncation=True,return_tensors='pt')
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self,idx):
        return self.input_ids[idx],self.attention_mask[idx]